In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Mounted at /content/drive
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).'


In [ ]:
!pip install transformers

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup


In [ ]:
# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load pre-trained model and tokenizer
model_name = 'HooshvareLab/bert-base-parsbert-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

In [ ]:
# Load the dataset
dataset = pd.read_csv('/content/drive/MyDrive/archive.zip', delimiter='\t',usecols=['comment', 'label_id'],on_bad_lines='skip')

In [ ]:
# Define dataset class
class SnappfoodDataset(Dataset):
    def __init__(self, csv_file):
        self.dataframe = pd.read_csv(csv_file, delimiter='\t', usecols=['comment', 'label_id'], on_bad_lines='skip')
        self.sentences = self.dataframe['comment'].values
        self.labels = self.dataframe['label_id'].values - 1  # shift ratings to 0-4
        
    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        sentence = self.sentences[index]
        label = torch.tensor(self.labels[index])
        encoding = tokenizer(sentence, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        input_ids, attention_mask = encoding['input_ids'][0], encoding['attention_mask'][0]
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': label}

In [ ]:
# Load dataset and split into train and test sets
dataset = SnappfoodDataset('/content/drive/MyDrive/archive.zip')
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
# Define data loader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)
num_epochs = 5
num_training_steps = len(train_loader) * num_epochs
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Train model
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch in train_loader:
        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label']

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels.to(device))

        # Compute loss and backpropagation
        loss = outputs.loss
        loss.backward()

        # Accumulate running loss
        running_loss += loss.item()

        # Update model parameters
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    # Compute average training loss
    train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1} Training Loss: {train_loss:.4f}")

ValueError: ignored